In [1]:
# Это нужно, чтобы можно было работать с файлами в Django
import sys 
import os, django

# Домашний комп
if os.path.isdir("C:\\IT\\python\\django_sbx\\django_sbx\\django_sbx"):
    sys.path.append("C:\\IT\\python\\django_sbx\\django_sbx\\django_sbx")
# Рабочий комп 
elif os.path.isdir("C:\\Users\\rustem.diveev\\Desktop\\django_sbx\\django_sbx"):
    sys.path.append("C:\\Users\\rustem.diveev\\Desktop\\django_sbx\\django_sbx")
    
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "django_sbx.settings")
django.setup()

# Это нужно, чтобы разрешить синхронные операции в асинхронном окружении - ничего не понял
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

In [2]:
from rest_framework.renderers import JSONRenderer 
from rest_framework.parsers import JSONParser

from drf_tutorial.models import Snippet
from drf_tutorial.serializers import SnippetSerializer

Snippet.objects.all().delete()

snippet = Snippet(code='foo = "bar"\n')
snippet.save()

snippet = Snippet(code='print("hello, world")\n')
snippet.save()

Snippet.objects.all()

<QuerySet [<Snippet: Snippet object (6)>, <Snippet: Snippet object (7)>]>

In [3]:
# Формирует словарь только из тех полей, что перечислены у сериализатора 
# Перевод экземаляра модели в нативные типы данных Python 
serializer = SnippetSerializer(snippet)
serializer.data

{'id': 7, 'title': '', 'code': 'print("hello, world")\n', 'linenos': False, 'language': 'python', 'style': 'friendly'}

In [4]:
# Преобразование данных в JSON, завершение процесса сериализации  
# Возвращает экземпляр bytes 
content = JSONRenderer().render(serializer.data)
content

b'{"id":7,"title":"","code":"print(\\"hello, world\\")\\n","linenos":false,"language":"python","style":"friendly"}'

In [5]:
# Десериализация - аналогичная 
# Первым шагом поток (байтовая последовательность) парсится на нативные типы данных Python 
import io 

stream = io.BytesIO(content)
data = JSONParser().parse(stream)
data

{'id': 7,
 'title': '',
 'code': 'print("hello, world")\n',
 'linenos': False,
 'language': 'python',
 'style': 'friendly'}

In [6]:
# Затем словарь приводится к валидному экземпляру модели 
serializer = SnippetSerializer(data=data)
serializer.is_valid()

True

In [7]:
serializer.validated_data

OrderedDict([('title', ''),
             ('code', 'print("hello, world")'),
             ('linenos', False),
             ('language', 'python'),
             ('style', 'friendly')])

In [8]:
serializer.save()

<Snippet: Snippet object (8)>

In [9]:
# Можно сериализовывать QuerySet - для этого, добавляем флаг many=True в аргументы сериализатора 

"""
    Здесь кстати можно заметить что code искажается - возвращается hello world без line feed в третьей сущности
"""

serializer = SnippetSerializer(Snippet.objects.all(), many=True)
serializer.data

[OrderedDict([('id', 6), ('title', ''), ('code', 'foo = "bar"\n'), ('linenos', False), ('language', 'python'), ('style', 'friendly')]), OrderedDict([('id', 7), ('title', ''), ('code', 'print("hello, world")\n'), ('linenos', False), ('language', 'python'), ('style', 'friendly')]), OrderedDict([('id', 8), ('title', ''), ('code', 'print("hello, world")'), ('linenos', False), ('language', 'python'), ('style', 'friendly')])]